In [ ]:
'''
prompt:
Escreva código python que lê todos os arquivos PDF da pasta data/exams e converte eles para texto.
 A função parse_pdfs deve retornar uma lista de tuplas: o primeiro elemento da tupla é o nom do arquivo PDF e o segundo elemento é o texto de cada PDF. '''

'\nprompt:\nEscreva código python que lê todos os arquivos PDF da pasta data/exams e converte eles para texto.  A função parse_pdfs deve retornar uma lista onde cada elemento é o texto de cada PDF.\n'

In [ ]:
!pip install pymupdf




[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\pedro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import os
import fitz  # PyMuPDF

def parse_pdfs(pasta="data/exams"):
    resultados = []
    
    # Percorre todos os arquivos da pasta
    for nome_arquivo in os.listdir(pasta):
        if nome_arquivo.lower().endswith(".pdf"):
            caminho_pdf = os.path.join(pasta, nome_arquivo)
            try:
                with fitz.open(caminho_pdf) as doc:
                    texto = ""
                    for pagina in doc:
                        texto += pagina.get_text()
                resultados.append((nome_arquivo, texto))
            except Exception as e:
                print(f"Erro ao processar {nome_arquivo}: {e}")
    
    return resultados


In [ ]:
pdf_textos = parse_pdfs()
for i, (pdf_filename, texto) in enumerate(pdf_textos):
    print(f"\n--- PDF {i+1} ---\n")
    print(pdf_filename, texto[:50])  # Mostra os primeiros 500 caracteres de cada PDF


pdf_texto = pdf_textos[0]
assert pdf_texto[0] == "oab-1.pdf"


--- PDF 1 ---

oab-1.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 2 ---

oab-10.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 3 ---

oab-100.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 4 ---

oab-101.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 5 ---

oab-102.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 6 ---

oab-103.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 7 ---

oab-104.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 8 ---

oab-105.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 9 ---

oab-106.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 10 ---

oab-107.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 11 ---

oab-108.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 12 ---

oab-109.pdf Questões comentadas OAB/FGV
Selecione a Matéria
Se

--- PDF 13 ---

oab-11.pdf Questões comentadas OAB/

In [ ]:
'''
prompt:
Escreva código python que recebe uma string de texto e retorna uma lista de dicionários. 
Cada elemento da lista deve conter as chaves "materia", "questão", "prova" e "tema", em que o valor é o texto entre a chave e \n". 
Exemplo: \nMatéria: ÉTICA PR OFISSIONAL\nQuestão:  001\nProva: XV\nTema:  INCOMP ATIBILID ADE E IMPEDIMENT O\n

Depois, o "enunciado" está entre o que vem depois do Tema e A). Leia A), B), C) e D) como entradas distintas no dicionário.

Em cada alternativa, há o texto "Resposta errada" ou "Resposta correta". Cada alternativa deve ter 3 sub-chaves: o texto, um booleano que diz se a resposta é correta, e a a justificativa que vem a seguir.

'''

'\nprompt:\nEscreva código python que recebe uma string de texto e retorna uma lista de dicionários. \nCada elemento da lista deve conter as chaves "materia", "questão", "prova" e "tema", em que o valor é o texto entre a chave e \n". \nExemplo: \nMatéria: ÉTICA PR OFISSIONAL\nQuestão:  001\nProva: XV\nTema:  INCOMP ATIBILID ADE E IMPEDIMENT O\n\n\nDepois, o "enunciado" está entre o que vem depois do Tema e A). Leia A), B), C) e D) como entradas distintas no dicionário.\n\nEm cada alternativa, há o texto "Resposta errada" ou "Resposta correta". Cada alternativa deve ter 3 sub-chaves: o texto, um booleano que diz se a resposta é correta, e a a justificativa que vem a seguir.\n\n'

In [ ]:
import re

def parse_questoes(pdf_filename, texto):

    padrao_bloco = re.compile(r'\nMatéria: (.*?)\nQuestão: (.*?)\nProva: (.*?)\nTema: (.*?)\n', re.DOTALL)
    blocos = list(padrao_bloco.finditer(texto))
    questoes = []

    for i, bloco in enumerate(blocos):
        materia = bloco.group(1).strip()
        questao = bloco.group(2).strip()
        prova = bloco.group(3).strip()
        tema = bloco.group(4).strip()

        # Determinar os limites do bloco de questão
        inicio_enunciado = bloco.end()
        fim_bloco = blocos[i + 1].start() if i + 1 < len(blocos) else len(texto)
        conteudo_questao = texto[inicio_enunciado:fim_bloco]

        # Separar enunciado e alternativas
        alternativas = re.split(r'([A-D]\))', conteudo_questao)
        enunciado = alternativas[0].strip()
        alternativas_dict = {}

        for j in range(1, len(alternativas) - 1, 2):
            letra = alternativas[j].strip(')')
            corpo = alternativas[j + 1].strip()

            # Extrair texto, resposta e justificativa
            match = re.match(r'^(.*?)(Resposta (correta|errada))(.*)$', corpo, re.DOTALL)
            if match:
                texto_alt = match.group(1).strip()
                correta = match.group(3).strip().lower() == 'correta'
                justificativa = match.group(4).strip()
            else:
                texto_alt = corpo
                correta = None
                justificativa = ''

            alternativas_dict[letra] = {
                'texto': texto_alt,
                'correta': correta,
                'justificativa': justificativa
            }

        questoes.append({
            'pdf_filename': pdf_filename,
            'materia': materia,
            'questao': questao,
            'prova': prova,
            'tema': tema,
            'enunciado': enunciado,
            'alternativas': alternativas_dict
        })

    return questoes


### TESTES ####


pdf_file_texto_1 = pdf_textos[0]


questoes = parse_questoes(pdf_file_texto_1[0], pdf_file_texto_1[1])

assert len(questoes) == 10

questao = questoes[0]

assert questao['materia'] == 'ÉTICA PROFISSIONAL'
assert questao['prova'] == 'XV'
assert questao['questao'] == '001'

assert questao['enunciado'].startswith("Abelardo")

assert questao['alternativas']['A']['correta'] == False
assert questao['alternativas']['B']['correta'] == False
assert questao['alternativas']['C']['correta'] == True
assert questao['alternativas']['D']['correta'] == False



questao

{'pdf_filename': 'oab-1.pdf',
 'materia': 'ÉTICA PROFISSIONAL',
 'questao': '001',
 'prova': 'XV',
 'tema': 'INCOMPATIBILIDADE E IMPEDIMENTO',
 'enunciado': 'Abelardo é magistrado vinculado ao Tribunal de Justiça do Estado K e requer licença\npara tratamento de questões particulares, pelo prazo de três anos, o que foi deferido.\nComo, antes de assumir o referido cargo, era advogado regularmente inscrito nos\nquadros da OAB, requer o seu reingresso, comprovando o afastamento das funções\njudicantes.\nNos termos do Estatuto da Advocacia, assinale a afirmativa correta.',
 'alternativas': {'A': {'texto': 'A incompatibilidade com a advocacia persiste mesmo após aposentadoria do\ncargo efetivo.',
   'correta': False,
   'justificativa': ': Afirmação falsa. Após a aposentadoria do juiz, não há mais impedimento para\no exercício da advocacia, já que não existe qualquer vedação no EAOAB. A única vedação que\nexiste é o exercício da advocacia no juízo/tribunal no qual atuou pelo prazo de três an

In [ ]:
'''
prompt:
Tenho uma lista de dicionários. Cada dicionário tem uma chave "prova", "materia", "questao" e "enunciado". 
Gere  código que produz um pandas dataframe salva em data/oab.csv
'''

'\nprompt:\nTenho uma lista de dicionários. Cada dicionário tem uma chave "prova", "materia", "questao" e "enunciado". \nGere  código que produz um pandas dataframe salva em data/oab.csv\n'

In [ ]:
import pandas as pd

def salvar_provas(lista_de_questoes):
    # Cria o DataFrame
    df = pd.DataFrame(lista_de_questoes)

    # Garante a ordem das colunas
    colunas_desejadas = ["prova", "materia", "questao", "enunciado"]
    df = df[colunas_desejadas]


    return df

# Supondo que cada elemento de pdf_textos seja uma lista de dicionários
todas_questoes = []

for pdf_texto in pdf_textos:
    questoes = parse_questoes(pdf_texto[0], pdf_texto[1])
    todas_questoes.extend(questoes)  # Adiciona todas as questões ao acumulador

    print(todas_questoes)

df = salvar_provas(todas_questoes)
df.to_csv("data/oab.csv", index=False, encoding="utf-8")
print(f"Arquivo salvo em data/oab.csv")

TypeError: parse_questoes() missing 1 required positional argument: 'texto'

In [ ]:
df['materia'].value_counts().head(25)

materia
ÉTICA PR OFISSIONAL                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             